In [1]:
import shutil

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import itertools
import shutil
import random
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [3]:
dir_path = 'D:/ratan/dog breeds/Images'

dir_path = 'D:/ratan/dog breeds/Images'
#print(os.listdir(dir_path))
for file in os.listdir(dir_path):
  new = os.rename(dir_path + '/' + file, dir_path + '/' + file.split('-')[1])
  
for file in os.listdir(dir_path):
  print(file, ':', len(os.listdir(dir_path + '/' + file)))

In [4]:
for file in os.listdir(dir_path):
  j = 1
  for i in os.listdir(dir_path + '/' + file):
    os.rename(dir_path + '/' + file  + '/' + i, dir_path + '/' + file + '/' + (file + '_' + str(j)) + '.jpg')
    j += 1

In [5]:

os.mkdir('D:/ratan/dog breeds/train')
os.mkdir('D:/ratan/dog breeds/valid')
os.mkdir('D:/ratan/dog breeds/test')


In [6]:
for file in os.listdir(dir_path):
  os.mkdir('D:/ratan/dog breeds/train' + '/' + file)
  os.mkdir('D:/ratan/dog breeds/valid' + '/' + file)
  os.mkdir('D:/ratan/dog breeds/test' + '/' + file)

os.chdir(dir_path)
for file in os.listdir(dir_path):
  for c in random.sample(glob.glob(file + '/' + file + '*'), 30):
    shutil.move(c, '/content/dog_breed/train' + '/' + file)
  for c in random.sample(glob.glob(file + '/' + file + '*'), 5):
    shutil.move(c, '/content/dog_breed/test' + '/' + file)
  for c in random.sample(glob.glob(file + '/' + file + '*'), 8):
    shutil.move(c, '/content/dog_breed/valid' + '/' + file)

In [7]:
os.chdir(dir_path)
for file in os.listdir(dir_path):
  total = len(os.listdir(dir_path + '/' + file))
  for c in random.sample(glob.glob(file + '/' + file + '*'), round(0.2*total)):
    shutil.move(c, 'D:/ratan/dog breeds/train' + '/' + file)
  for c in random.sample(glob.glob(file + '/' + file + '*'), round(0.02*total)):
    shutil.move(c, 'D:/ratan/dog breeds/test' + '/' + file)
  for c in random.sample(glob.glob(file + '/' + file + '*'), round(0.02*total)):
    shutil.move(c, 'D:/ratan/dog breeds/valid' + '/' + file)

In [8]:
train_dir = 'D:/ratan/dog breeds/train'
test_dir = 'D:/ratan/dog breeds/test'
valid_dir = 'D:/ratan/dog breeds/valid'

train_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = train_path, target_size = (224, 224), batch_size = 10)
test_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = test_path, target_size = (224, 224), batch_size = 10)
valid_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet.preprocess_input).flow_from_directory(directory = valid_path, target_size = (224, 224), batch_size = 10)

imgs, labels = next(train_batches)

def plot_images(image_arr):
  fig, axes = plt.subplots(1, 10, figsize = (20, 20))
  axes = axes.flatten()
  for image, ax in zip(image_arr, axes):
    ax.imshow(image)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

plot_images(imgs)
print(labels)

In [15]:
IMG_SIZE = (224, 224)

def create_data_loaders(train_dir, valid_dir, test_dir, image_size=IMG_SIZE):
  """
  Creates a training and test image BatchDataset from train_dir and test_dir.
  """
  train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  valid_data = tf.keras.preprocessing.image_dataset_from_directory(valid_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  # Note: the test data is the same as the previous experiment, we could
  # skip creating this, but we'll leave this here to practice.
  test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                  label_mode="categorical",
                                                                  image_size=image_size)
  
  return train_data, test_data

In [10]:
# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential([
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNetB0
], name ="data_augmentation")

In [12]:
INPUT_SHAPE = (224, 224, 3)
BASE_MODEL = tf.keras.applications.EfficientNetB0(include_top=False)

def create_model(input_shape=INPUT_SHAPE, base_model=BASE_MODEL, num_classes=10):
  # Fine-tune?
  base_model.trainable = False

  # Create input layer
  inputs = layers.Input(shape=input_shape, name="input_layer")

  # Add in data augmentation Sequential model as a layer
  x = data_augmentation(inputs)

  # Give base_model inputs (after augmentation) and don't train it
  x = base_model(x, training=False)

  # Pool output features of base model
  x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)

  # Put a dense layer on as the output
  outputs = layers.Dense(num_classes, activation="softmax", name="output_layer")(x)

  # Make a model with inputs and outputs
  model = keras.Model(inputs, outputs)

  # Compile the model
  model.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  
  return model

16705208/16705208 [==============================] - 5s 0us/step


In [13]:
def load_and_prep_image(filename, img_shape=224, scale=False):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  # Rescale the image (get all values between 0 and 1)
  if scale:
    return img/255.
  else:
    return img

In [16]:
train_data, valid_data = create_data_loaders(train_dir=train_dir, valid_dir = valid_dir, test_dir = test_dir)

Found 4111 files belonging to 120 classes.
Found 406 files belonging to 120 classes.
Found 406 files belonging to 120 classes.


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Conv2D, Flatten, MaxPool2D, BatchNormalization, Dropout, Softmax
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.metrics import confusion_matrix
from keras.applications.mobilenet import decode_predictions

In [17]:
model_1 = create_model(num_classes=len(train_data.class_names))
model_1.summary()
# Fit the model
history_1_percent = model_1.fit(train_data,
                    epochs=5,
                    steps_per_epoch=len(train_data),
                    validation_data=valid_data,
                    validation_steps=int(0.25 * len(valid_data)), # validate for less steps
                    # Track model training logs
                    verbose = 1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling_laye  (None, 1280)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 output_layer (Dense)        (None, 120)               153720    
                                                                 
Total params: 4,203,291
Trainable params: 153,720
Non-trainab

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'], )

history = model.fit(x= train_batches, validation_data= valid_batches, epochs = 1, verbose = 1)

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show(history)

pred = model.predict(test_batches)

label = decode_predictions(pred)

pred = np.argmax(pred, axis = -1)

In [18]:
class_names = train_data.class_names
class_names

['Afghan_hound',
 'African_hunting_dog',
 'Airedale',
 'American_Staffordshire_terrier',
 'Appenzeller',
 'Australian_terrier',
 'Bedlington_terrier',
 'Bernese_mountain_dog',
 'Blenheim_spaniel',
 'Border_collie',
 'Border_terrier',
 'Boston_bull',
 'Bouvier_des_Flandres',
 'Brabancon_griffon',
 'Brittany_spaniel',
 'Cardigan',
 'Chesapeake_Bay_retriever',
 'Chihuahua',
 'Dandie_Dinmont',
 'Doberman',
 'English_foxhound',
 'English_setter',
 'English_springer',
 'EntleBucher',
 'Eskimo_dog',
 'French_bulldog',
 'German_shepherd',
 'German_short',
 'Gordon_setter',
 'Great_Dane',
 'Great_Pyrenees',
 'Greater_Swiss_Mountain_dog',
 'Ibizan_hound',
 'Irish_setter',
 'Irish_terrier',
 'Irish_water_spaniel',
 'Irish_wolfhound',
 'Italian_greyhound',
 'Japanese_spaniel',
 'Kerry_blue_terrier',
 'Labrador_retriever',
 'Lakeland_terrier',
 'Leonberg',
 'Lhasa',
 'Maltese_dog',
 'Mexican_hairless',
 'Newfoundland',
 'Norfolk_terrier',
 'Norwegian_elkhound',
 'Norwich_terrier',
 'Old_English_she

pred

test_batches.class_indices

test_batches.classes

from sklearn.metrics import accuracy_score

accuracy_score(pred, test_batches.classes)

import tensorflow as tf
from tensorflow.keras.applications import imagenet_utils
result = imagenet_utils.decode_predictions(pred)

In [19]:
img = load_and_prep_image("C:/Users/User PC/Desktop/123.jpg")
img

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[ 53.979595 ,  60.214287 ,   6.214286 ],
        [ 57.224487 ,  62.224487 ,   7.795917 ],
        [ 59.08673  ,  62.08673  ,   8.65816  ],
        ...,
        [138.1986   , 148.1986   ,  77.19859  ],
        [137.648    , 147.648    ,  76.648    ],
        [137.       , 147.42822  ,  76.21411  ]],

       [[ 49.280613 ,  56.280613 ,   4.280612 ],
        [ 54.22959  ,  58.22959  ,   7.2295923],
        [ 55.642857 ,  59.642857 ,   8.642858 ],
        ...,
        [137.95418  , 147.95418  ,  76.95417  ],
        [137.77039  , 147.77039  ,  76.770386 ],
        [137.71933  , 148.14755  ,  76.93343  ]],

       [[ 48.87245  ,  56.87245  ,   5.872451 ],
        [ 50.714287 ,  57.714287 ,   6.714287 ],
        [ 50.92857  ,  57.92857  ,   6.92857  ],
        ...,
        [138.20877  , 148.20877  ,  75.20878  ],
        [140.23973  , 150.23973  ,  77.23973  ],
        [137.94386  , 147.94386  ,  76.94386  ]],

       ...,

     

In [20]:
expanded = tf.expand_dims(img, axis=0) # expand image dimensions (224, 224, 3) -> (1, 224, 224, 3)
pred = model_1.predict(expanded)
pred

1/1 [==============================] - 3s 3s/step


array([[1.36938776e-04, 1.39096592e-04, 2.61792418e-04, 6.53315242e-03,
        8.09192308e-04, 2.21443224e-05, 6.78232391e-05, 1.76869125e-05,
        1.35027340e-05, 2.74258109e-05, 1.82349072e-03, 5.53726295e-06,
        2.86477571e-05, 1.33318943e-03, 7.16077862e-04, 1.65872858e-04,
        1.75972991e-02, 3.81219666e-04, 1.52644789e-04, 1.20712015e-04,
        1.29981153e-03, 5.84774127e-04, 7.07322943e-06, 1.01814359e-04,
        5.79978514e-04, 3.69846704e-04, 5.26519318e-04, 2.35891843e-04,
        6.94224145e-05, 5.15761180e-03, 4.81878582e-04, 4.59183793e-04,
        1.86408201e-04, 1.12814130e-04, 4.06467792e-04, 4.72892716e-05,
        1.38650794e-04, 2.97934403e-05, 2.07548692e-05, 8.78432729e-06,
        7.29226649e-01, 2.02203562e-04, 2.48255744e-03, 9.33699775e-05,
        1.92005173e-05, 1.59977608e-05, 5.13698731e-04, 1.01637689e-03,
        4.66044949e-05, 3.09260213e-04, 6.27199624e-05, 1.35445400e-04,
        7.27606937e-04, 2.07219000e-05, 3.40403877e-02, 6.830113

In [21]:
class_names[tf.argmax(pred[0])]

'Labrador_retriever'

In [26]:
model_1.save("120_dog_breeds.hdf5")

INFO:tensorflow:Assets written to: C:/Users/User PC\assets


INFO:tensorflow:Assets written to: C:/Users/User PC\assets
